In [1]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hw').getOrCreate()

In [2]:
# read the csv file
df = spark.read.csv('/FileStore/tables/walmart_stock.csv', inferSchema=True, header=True)

In [3]:
# how does the schema look like
df.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [4]:
# print all column names
df.columns

Out[30]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [5]:
# print the first 5 records
for i in df.head(5):
  print(i)
  print('\n')

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)

In [6]:
# describe the dataframe
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Date| Open| High| Low| Close| Volume| Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258| 1258|
 mean| null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
 stddev| null| 6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991| 4519780.8431556|6.722609449996857|
 min|2012-01-03|56.389998999999996| 57.060001| 56.299999| 56.419998| 2094900| 50.363689|
 max|2016-12-30| 90.800003| 90.970001| 89.25| 90.470001| 80898100|84.91421600000001|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [7]:
df.describe().printSchema()

root
-- summary: string (nullable = true)
-- Date: string (nullable = true)
-- Open: string (nullable = true)
-- High: string (nullable = true)
-- Low: string (nullable = true)
-- Close: string (nullable = true)
-- Volume: string (nullable = true)
-- Adj Close: string (nullable = true)

In [8]:
from pyspark.sql.functions import format_number

In [9]:
res = df.describe()

In [10]:
res.select(res['summary'], format_number(res['Open'].cast('float'),2).alias('Open'),
format_number(res['High'].cast('float'),2).alias('Open'),
format_number(res['Low'].cast('float'),2).alias('Open'),
format_number(res['Close'].cast('float'),2).alias('Open'),
res['Volume'].cast('int').alias('Volume')).show()

+-------+--------+--------+--------+--------+--------+
summary| Open| Open| Open| Open| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

In [11]:
# create new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

df2 = df.withColumn('HV Ratio', df['High']/df['Volume'])
df2.select('HV Ratio').show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714653321546E-6|
6.290848613094555E-6|
4.669412994783916E-6|
7.367338463826307E-6|
8.915604778943901E-6|
8.644477436914568E-6|
9.351828421515645E-6|
 8.29141562102703E-6|
7.712212102001476E-6|
7.071764823529412E-6|
1.015495466386981E-5|
6.576354146362592...|
 5.90145296180676E-6|
8.547679455011844E-6|
8.420709512685392E-6|
1.041448341728929...|
8.316075414862431E-6|
9.721183814992126E-6|
8.029436027707578E-6|
6.307432259386365E-6|
+--------------------+
only showing top 20 rows

In [12]:
# which day had the highest price
df.orderBy(df['High'].desc()).head(1)[0][0]

Out[38]: '2015-01-13'

In [13]:
# mean of the close column 
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
 avg(Close)|
+-----------------+
72.38844998012726|
+-----------------+

In [14]:
# max and min of the volume column
from pyspark.sql.functions import max,min


In [15]:
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
max(Volume)|min(Volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

In [16]:
# how many days was the close lower than 60 dollars
df.filter('Close < 60').count()

Out[43]: 81

In [17]:
from pyspark.sql.functions import count

In [18]:
res = df.filter(df['Close']<60)


In [19]:
res.select(count('Close')).show()

+------------+
count(Close)|
+------------+
 81|
+------------+

In [20]:
# what percentage of the time was the high greater than 80 dollars
(df.filter(df['High']>80).count() * 1.0/df.count()) * 100

Out[48]: 9.141494435612083

In [21]:
# what is pearson's correlation between High and Volume
from pyspark.sql.functions import corr
df.select(corr('High','Volume')).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061737161|
+-------------------+

In [22]:
# what's the max high per year?
from pyspark.sql.functions import year
yeardf = df.withColumn('Year', year(df['Date']))
max_df = yeardf.groupBy('Year').max()
max_df.select('Year', 'max(High)').show()

+----+---------+
Year|max(High)|
+----+---------+
2015|90.970001|
2013|81.370003|
2014|88.089996|
2012|77.599998|
2016|75.190002|
+----+---------+

In [23]:
# avg close for each calendar month
from pyspark.sql.functions import month
monthdf = df.withColumn('Month', month('Date'))
monthavgs = monthdf.select(['Month', 'Close']).groupBy('Month').mean()
monthavgs.select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1|71.44801958415842|
 2| 71.306804443299|
 3|71.77794377570092|
 4|72.97361900952382|
 5|72.30971688679247|
 6| 72.4953774245283|
 7|74.43971943925233|
 8|73.02981855454546|
 9|72.18411785294116|
 10|71.57854545454543|
 11| 72.1110893069307|
 12|72.84792478301885|
+-----+-----------------+